# Machine Learning for Dynamical Systems 

## Artificial Neural Networks: A quick summary

Artificial neural networks (ANNs), in the form of multilayer perceptrons (MLPs) are:

* Networks made up of a chain of several layers: $$f(\mathbf x) = f^{(3)}(f^{(2)}(f^{(1)}(\mathbf x; \theta^{(1)}); \theta^{(2)}); \theta^{(3)}),$$ where each layers is a parametrized nonlinear transformation: $$\sigma(\mathbf z^{\mathrm T} \mathbf W + \mathbf b),$$ where $\theta=\{\mathbf{W},\mathbf{b}\}$ are the learnable parameters and $\sigma$ is a nonlinear activation function, such as $\tanh$
* They are *universal function approximators* with many parameters $\theta$

* Given training data, we seek the best set of parameters by minimzing a loss function $L(\theta)$, e.g. a mean square error, on that training data, by means of a gradient descent optimization, for which we need $$\nabla_\theta L(\mathbf \theta).$$ 
* The gradients are computed via *backpropagation*, i.e. chain rule of derivatives
* ML libaries compute this via an automatic differenation system, that is able to systematically track all elementary operations performed
* The parameters are then updated with some form of gradient descent 
$$\theta_{new} = \theta - \eta \nabla_\theta L(\mathbf \theta)$$, 
with some learning rate $\eta$

### The Key Ingredients 

* Training data 
* A parametrized model 
* A loss function 
* The ability to take gradients of the loss function, to update the parameters of the model

## ML for Dynamical System 

How can you ML methods like ANN for dynamical sytems? 

In [ ]:
### Apprixmate a DS 



In [ ]:
### Prior Knowledge 

## Methods for this Workshop 

* Neural Differential Equations 
* Reservoir Computing 
* Symbolic Methods (SINDy) 

## Neural Differential Equations 



## Reservoir Computing 

## Symbolic Methods (SINDy)

In [ ]:
## Projects

